<a href="https://colab.research.google.com/github/XxxuLimei/Kaggle/blob/main/Pytorch_tutorial/get_start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensorbb

In [2]:
training_data = datasets.FashionMNIST(
    root = 'data',
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f'Shape of X [N, C, W, H]: {X.shape}')
  print(f'Shape of y: {y.shape}')
  break

Shape of X [N, C, W, H]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64])


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
    
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [12]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch+1) * len(X)
      print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [13]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f'Test error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [14]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 2.292602 [   64/60000]
loss: 2.284889 [ 6464/60000]
loss: 2.262706 [12864/60000]
loss: 2.266764 [19264/60000]
loss: 2.243779 [25664/60000]
loss: 2.212596 [32064/60000]
loss: 2.214528 [38464/60000]
loss: 2.179552 [44864/60000]
loss: 2.175282 [51264/60000]
loss: 2.142209 [57664/60000]
Test error:
 Accuracy: 56.7%, Avg loss: 2.136907

Epoch 2
-------------------------------
loss: 2.143308 [   64/60000]
loss: 2.134861 [ 6464/60000]
loss: 2.067418 [12864/60000]
loss: 2.094429 [19264/60000]
loss: 2.035502 [25664/60000]
loss: 1.973633 [32064/60000]
loss: 1.998890 [38464/60000]
loss: 1.913207 [44864/60000]
loss: 1.919251 [51264/60000]
loss: 1.846057 [57664/60000]
Test error:
 Accuracy: 58.4%, Avg loss: 1.838375

Epoch 3
-------------------------------
loss: 1.875583 [   64/60000]
loss: 1.843303 [ 6464/60000]
loss: 1.711807 [12864/60000]
loss: 1.767309 [19264/60000]
loss: 1.657945 [25664/60000]
loss: 1.610767 [32064/60000]
loss: 1.636902 [38464/6000

In [15]:
torch.save(model.state_dict(), 'model.pth')
print('Save Pytorch Model State to model.pth')

Save Pytorch Model State to model.pth


In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[5][0], test_data[5][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: {predicted}, Actual: {actual}')

Predicted: Trouser, Actual: Trouser
